In [4]:
# torch packages
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.functional import pad
from torch import Tensor
from torch.utils.data import DataLoader

import torchtext
import torchtext.datasets as datasets
from torchtext.data.functional import to_map_style_dataset
from torchtext.vocab import build_vocab_from_iterator

from datasets import load_dataset
from transformer_from_scratch.transformer_instantiate import transformer

# importing required libraries
import os
import math
import copy
import time
import spacy
import jieba
import numpy as np
import pandas as pd
import random
import portalocker

# visualization packages
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from mpl_toolkits import mplot3d
from tqdm import tqdm

### Hyper parameters

In [23]:
MAX_PADDING = 20
MAX_SEQ_LEN = 50
BATCH_SIZE = 128
D_MODEL = 256
D_FFN = 512
N_HEADS = 8
N_LAYERS = 3
N_EPOCHS = 10
DROPOUT = 0.1
LEARNING_RATE = 1e-4
CLIP = 1
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
# url: https://huggingface.co/datasets/Helsinki-NLP/opus-100/tree/main/en-zh
# dataset = load_dataset("Helsinki-NLP/opus-100", "en-zh")
en_zh_dataset = load_dataset("parquet", data_files={
    'train': 'datasets/opus-100/en-zh/train-00000-of-00001.parquet',
    'validation': 'datasets/opus-100/en-zh/validation-00000-of-00001.parquet'
    'test': 'datasets/opus-100/en-zh/test-00000-of-00001.parquet',
    }
)

In [7]:
pd.DataFrame(en_zh_dataset['train']['translation'][:100])

,en,zh
0,Sixty-first session,第六十一届会议
1,I took some medicine for my mu for my mu my mu...,减轻酸... 酸痛的药 减轻酸痛的药
2,It's a challenge. God is challenging you. He's...,上帝在挑战你，他说你是笨蛋
3,"Oh, baby.",.. 寶貝
4,- Lucinda?,- 盧辛達？
...,...,...
95,Take 'em all.,通通拿下
96,I so knowthat. That's what's great.,一点都没错，我很高兴
97,Harlend thinks you're some kind of criminal ma...,哈兰德 感谢你的罪案策划
98,"In short, the US has moved to bring together a...",简言之，美国正在试图团结该地区所有担心中国以邻为壑的贸易和汇率政策的国家。对美国来说，其他八...


### Loading the Tokenizers

In [8]:
def load_tokenizers():
    """
    Load the English tokenizers provided by spaCy

    Returns:
        spacy_en: English tokenizer
    """
    try:
        spacy_en = spacy.load("en_core_web_sm")
    except OSError:
        os.system("python -m spacy download en_core_web_sm")
        spacy_en = spacy.load("en_core_web_sm")

    print("Loaded English tokenizer.")
    return spacy_en

In [9]:
spacy_en = load_tokenizers()

Loaded English tokenizer.


### Tokenize the Sequences

In [10]:
def tokenize(text, language):
    """
    Split a string into its tokens using the corresponding tokenizer.

    Args:
        text:     string
        language: en or zh    
    Returns:
        tokenized list of strings 
    """
    if language == 'en':
        return [tok.text.lower() for tok in spacy_en.tokenizer(text)]
    elif language == 'zh':
        return list(jieba.cut(text))

### Yield Tokens

In [11]:
def yield_tokens(sentences, language):
    """
    Return the tokens for the appropriate language.

    Args:
        sentences: List of sentences to tokenize.
        language:  The language code ('en' for English, 'zh' for Chinese).

    Yields:
        sequences based on index
    """
    for sentence in sentences:
        yield tokenize(sentence, language)

### Building the Vocabulary

In [12]:
def build_vocabulary(dataset, language, min_freq=2):
    """
    Builds a vocabulary from a specified language across all splits (train, test, validation) of a dataset.

    Args:
        dataset:  The dataset object containing the 'train', 'test', and 'validation' splits. Each split should contain a 'translation' field with subfields for each language (e.g., 'en' for English, 'zh' for Chinese).
        language: The language code ('en' for English, 'zh' for Chinese) for which the vocabulary will be built. This code should match the subfields in the 'translation' field of the dataset.
        min_freq: The minimum frequency a token must have across the dataset to be included in the vocabulary. Tokens appearing fewer times than this threshold will be excluded. Defaults to 2.

    Returns:
        Vocab: A vocabulary object that maps tokens to indices for the specified language. This vocabulary includes special tokens
               ("<unk>", "<pad>", "<bos>", "<eos>") and all tokens that appear at least 'min_freq' times in the dataset.
    """

    all_sentences = []
    for split in ['train', 'validation', 'test']:
        all_sentences.extend([item['translation'][language] for item in dataset[split]])
    
    vocab = build_vocab_from_iterator(
        yield_tokens(all_sentences, language),
        specials=["<bos>", "<eos>", "<pad>", "<unk>"],
        min_freq=min_freq
    )

    vocab.set_default_index(vocab["<unk>"])

    return vocab

### Load the Vocabulary

In [13]:
def load_vocab(dataset, min_freq=2):
    """
    Load or build the vocabulary for English and Chinese languages.

    Args:
        dataset: The dataset objet containing the 'train', 'test', and 'validation' splits
        min_freq: minimum frequency needed to include a word in the vocabulary
    
    Returns:
        src_vocab: English vocabulary
        trg_vocab: Chinese vocabulary  
    """
    vocab_filename = "checkpoints/vocab.pt"
    if not os.path.exists(vocab_filename):
        # build the English/Chinese vocabulary if it does not exist
        src_vocab = build_vocabulary(dataset, "en", min_freq)
        trg_vocab = build_vocabulary(dataset, "zh", min_freq)
        
        torch.save((src_vocab, trg_vocab), vocab_filename)
    else:
        # load the vocab if it exists
        src_vocab, trg_vocab = torch.load(vocab_filename)

    print("Vocabulary loaded.\nSource vocab size:", len(src_vocab), "\nTarget vocab size:", len(trg_vocab))
    return src_vocab, trg_vocab

In [14]:
src_vocab, trg_vocab = load_vocab(en_zh_dataset)

Vocabulary loaded.
Source vocab size: 81674 
Target vocab size: 122394


### Indexing Sequences

In [15]:
def data_process(raw_data):
    """
    Process raw sentences by tokenizing and converting to integers based on the vocabulary.

    Args:
        raw_data:  English-Chinese sentence pairs
        src_vocab: English vocabulary
        trg_vocab: Chinese vocabulary 
    
    Returns:
        data: tokenized data converted to index based on vocabulary
    """
    data = []
    # loop through each sentence pair
    for item in tqdm(raw_data, desc='Processing'):

        # tokenize the sentence and convert each word to an integers
        src_tensor = torch.tensor([src_vocab[token] for token in tokenize(item['translation']['en'], 'en')], dtype=torch.long) 
        trg_tensor = torch.tensor([trg_vocab[token] for token in tokenize(item['translation']['zh'], 'zh')], dtype=torch.long)

        data.append((src_tensor, trg_tensor))
    return data

### Generating Batches

In [16]:
BOS_IDX = trg_vocab['<bos>']
EOS_IDX = trg_vocab['<eos>']
PAD_IDX = trg_vocab['<pad>']

In [17]:
def generate_batch(data_batch):
    """
    Process indexed-sequences by adding <bos>, <eos>, and <pad> tokens.

    Args:
        data_batch: English-Chinese indexed-sentence pairs

    Returns:
        two batches: one for English and one for Chinese
    """
    src_batch, trg_batch = [], []

    for (src_item, trg_item) in data_batch:
        # add <bos> and <eos> indices before and after sentence
        src_item = torch.cat([torch.tensor([BOS_IDX]), src_item, torch.tensor([EOS_IDX])], dim=0).to(device)
        trg_item = torch.cat([torch.tensor([BOS_IDX]), trg_item, torch.tensor([EOS_IDX])], dim=0).to(device)

        # add padding
        src_batch.append(pad(
            src_item,
            (0, MAX_PADDING - len(src_item)),
            value=PAD_IDX
        ))

        trg_batch.append(pad(
            trg_item,
            (0, MAX_PADDING - len(trg_item)),
            value=PAD_IDX
        ))

    return torch.stack(src_batch), torch.stack(trg_batch)

### Preprocessing the Data

In [18]:
train_data = data_process(en_zh_dataset['train'])
val_data = data_process(en_zh_dataset['validation'])
test_data = data_process(en_zh_dataset['test'])

Processing:   0%|          | 0/1000000 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\IVVIMU\AppData\Local\Temp\jieba.cache
Loading model cost 0.616 seconds.
Prefix dict has been built successfully.
Processing: 100%|██████████| 2000/2000 [00:00<00:00, 3106.36it/s]


In [19]:
train_iter = DataLoader(
    to_map_style_dataset(train_data),
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=True,
    collate_fn=generate_batch
)
val_iter = DataLoader(
    to_map_style_dataset(val_data),
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=True,
    collate_fn=generate_batch
)
test_iter = DataLoader(
    to_map_style_dataset(test_data),
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=True,
    collate_fn=generate_batch
)

### Creating the Model

In [21]:
model = transformer(
    src_vocab=src_vocab,
    trg_vocab=trg_vocab,
    d_model=D_MODEL,
    d_ffn=D_FFN,
    n_heads=N_HEADS,
    n_layers=N_LAYERS,
    device=device,
    dropout=DROPOUT,
    max_seq_len=MAX_SEQ_LEN
).to(device)

In [22]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 87,650,330 trainable parameters


### Training functions

In [24]:
def train(model, iterator, optimizer, criterion, clip):
    """
    Train the model on the given data.

    Args:
        model:     Transformer model to be trained
        iterator:  data to be trained on
        optimizer: optimizer for updating parameters
        criterion: loss function for updating parameters
        clip:      value to help prevent exploding gradients

    Returns:
        loss for epoch
    """

    # set the model to training mode
    model.train()

    epoch_loss = 0
    # loop through each batch in the iterator
    for i, batch in enumerate(iterator):
        # set the source and target batches
        src, trg = batch

        # zero the gradients
        optimizer.zero_grad()

        # logits for each output
        logits = model(src, trg[:, :-1])

        # expected output
        expected_output = trg[:, 1:]

        # calculate the loss
        loss = criterion(
            logits.contiguous().view(-1, logits.shape[-1]),
            expected_output.contiguous().view(-1)
        )

        # backpropagation
        loss.backward()

        # clip the weights
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # update the loss
        optimizer.step()

        # update the loss
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [27]:
def evaluate(model, iterator, criterion):
    """
    Evaluate the model on the given data.

    Args:
        model:     Transformer model to be trained
        iterator:  data to be evaluated
        criterion: loss function for assessing outputs

    Returns:
        loss for the data
    """

    # set the model to evaluation mode
    model.eval()

    epoch_loss = 0

    # evaluate without updating gradients
    with torch.no_grad():
        # loop through each batch in the iterator
        for i, batch in enumerate(iterator):

            # set the source and target batches
            src, trg = batch

            # logits for each output
            logits = model(src, trg[:, :-1])

            # expected output
            expected_output = trg[:, 1:]

            # calculate the loss
            loss = criterion(
                logits.contiguous().view(-1, logits.shape[-1]),
                expected_output.contiguous().view(-1)
            )

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [26]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

### Training the model

In [ ]:
best_valid_loss = float('inf')
model_path = 'checkpoints/transformer.pt'

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iter, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    # save the model when it performs better than the previous run
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), model_path)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

### Evaluating the model

In [ ]:
model.load_state_dict(torch.load(model_path))

test_loss = evaluate(model, test_iter, criterion)

print(f'Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f}')